In [40]:
import pandas as pd
import numpy as np
import torch
from autogluon.tabular import TabularPredictor
import os

In [41]:

print(os.getcwd())

/Users/zhaohongbo/PycharmProjects/Pytorch_learning/VC


In [42]:
df=pd.read_csv('../data/investments_VC_clean.csv')

In [43]:
df.head()
drop_lst=['permalink','homepage_url']

In [44]:
df.drop(columns=drop_lst, inplace=True)

In [45]:
"""
Data Wrangler 生成的单元格。
"""
def clean_data(df):
    # 将缺少的值替换为列: 'name' 中的 "Unknown"
    df = df.fillna({'name': "Unknown"})
    # 将缺少的值替换为列: 'category_list' 中的 "Unknown"
    df = df.fillna({'category_list': "Unknown"})
    # 将缺少的值替换为列: ' market ' 中的 "Unknown"
    df = df.fillna({' market ': "Unknown"})
    # 将缺少的值替换为列: 'status' 中的 "Unknown"
    df = df.fillna({'status': "Unknown"})
    # 将缺少的值替换为列: ' funding_total_usd ' 中的 "0"
    df = df.fillna({' funding_total_usd ': " -   "})
    return df

df_clean = clean_data(df.copy())
df_clean.head()

,name,category_list,market,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,...,secondary_market,product_crowdfunding,round_A,round_B,round_C,round_D,round_E,round_F,round_G,round_H
0,#waywire,|Entertainment|Politics|Social Media|News|,News,"17,50,000",acquired,USA,NY,New York City,New York,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,&TV Communications,|Games|,Games,"40,00,000",operating,USA,CA,Los Angeles,Los Angeles,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,'Rock' Your Paper,|Publishing|Education|,Publishing,"40,000",operating,EST,NaN,Tallinn,Tallinn,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,(In)Touch Network,|Electronics|Guides|Coffee|Restaurants|Music|iPhone|Apps|Mobile|iOS|E-Commerce|,Electronics,"15,00,000",operating,GBR,NaN,London,London,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-R- Ranch and Mine,|Tourism|Entertainment|Games|,Tourism,"60,000",operating,USA,TX,Dallas,Fort Worth,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
"""
Data Wrangler 生成的单元格。
"""
def clean_data(df_clean):
    # 将缺少的值替换为列: 'country_code' 中的 "Unknown"
    df_clean = df_clean.fillna({'country_code': "Unknown"})
    # 将缺少的值替换为列: 'state_code', 'region', 'city' 中的 "Unknown"
    df_clean = df_clean.fillna({'state_code': "Unknown", 'region': "Unknown", 'city': "Unknown"})
    df['seed_missing'] = df['seed'].isnull().astype(int)
    # 从公式创建的列 'seed_missing'
    df_clean['fund_info_missing'] = df_clean['seed'].isnull().astype(int)
    # 将缺少的值替换为列: 'venture'、'undisclosed'和其他列16 中的 0
    df_clean = df_clean.fillna({'venture': 0, 'undisclosed': 0, 'convertible_note': 0, 'debt_financing': 0, 'angel': 0, 'grant': 0, 'private_equity': 0, 'post_ipo_equity': 0, 'post_ipo_debt': 0, 'secondary_market': 0, 'round_A': 0, 'round_B': 0, 'round_C': 0, 'round_D': 0, 'round_E': 0, 'round_F': 0, 'round_G': 0, 'round_H': 0})
    # 将缺少的值替换为列: 'equity_crowdfunding', 'product_crowdfunding', 'seed' 中的 0
    df_clean = df_clean.fillna({'equity_crowdfunding': 0, 'product_crowdfunding': 0, 'seed': 0})
    return df_clean

df_clean_1 = clean_data(df_clean.copy())
df_clean_1.head()

,name,category_list,market,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,...,product_crowdfunding,round_A,round_B,round_C,round_D,round_E,round_F,round_G,round_H,fund_info_missing
0,#waywire,|Entertainment|Politics|Social Media|News|,News,"17,50,000",acquired,USA,NY,New York City,New York,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,&TV Communications,|Games|,Games,"40,00,000",operating,USA,CA,Los Angeles,Los Angeles,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,'Rock' Your Paper,|Publishing|Education|,Publishing,"40,000",operating,EST,Unknown,Tallinn,Tallinn,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,(In)Touch Network,|Electronics|Guides|Coffee|Restaurants|Music|iPhone|Apps|Mobile|iOS|E-Commerce|,Electronics,"15,00,000",operating,GBR,Unknown,London,London,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,-R- Ranch and Mine,|Tourism|Entertainment|Games|,Tourism,"60,000",operating,USA,TX,Dallas,Fort Worth,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [47]:
"""
Data Wrangler 生成的单元格
"""
def clean_data(df_clean_1):
    target_cols = [
    'seed',              # 种子轮融资
    'venture',           # 风险投资
    'equity_crowdfunding', # 股权众筹
    'undisclosed',       # 未披露融资
    'convertible_note',  # 可转换债券融资
    'debt_financing',    # 债务融资
    'angel',             # 天使轮融资
    'grant',             # 政府/机构赠款
    'private_equity',    # 私募股权投资
    'post_ipo_equity',   # 上市后股权融资
    'post_ipo_debt',     # 上市后债务融资
    'secondary_market',  # 二级市场交易
    'product_crowdfunding', # 产品众筹
    'round_A',           # A轮融资
    'round_B',           # B轮融资
    'round_C',           # C轮融资
    'round_D',           # D轮融资
    'round_E',           # E轮融资
    'round_F',           # F轮融资
    'round_G',           # G轮融资
    'round_H'            # H轮融资
]
    df_clean_1[target_cols] = df_clean_1[target_cols].apply(np.log1p)
    return df_clean_1

df_clean_2 = clean_data(df_clean_1.copy())
df_clean_2.head()

,name,category_list,market,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,...,product_crowdfunding,round_A,round_B,round_C,round_D,round_E,round_F,round_G,round_H,fund_info_missing
0,#waywire,|Entertainment|Politics|Social Media|News|,News,"17,50,000",acquired,USA,NY,New York City,New York,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,&TV Communications,|Games|,Games,"40,00,000",operating,USA,CA,Los Angeles,Los Angeles,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,'Rock' Your Paper,|Publishing|Education|,Publishing,"40,000",operating,EST,Unknown,Tallinn,Tallinn,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,(In)Touch Network,|Electronics|Guides|Coffee|Restaurants|Music|iPhone|Apps|Mobile|iOS|E-Commerce|,Electronics,"15,00,000",operating,GBR,Unknown,London,London,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,-R- Ranch and Mine,|Tourism|Entertainment|Games|,Tourism,"60,000",operating,USA,TX,Dallas,Fort Worth,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [48]:
df_final=df_clean_2.copy()

In [49]:
print(*df_final.columns,sep=',')
df_final['status'].value_counts()

name,category_list, market , funding_total_usd ,status,country_code,state_code,region,city,funding_rounds,founded_at,founded_month,founded_quarter,founded_year,first_funding_at,last_funding_at,seed,venture,equity_crowdfunding,undisclosed,convertible_note,debt_financing,angel,grant,private_equity,post_ipo_equity,post_ipo_debt,secondary_market,product_crowdfunding,round_A,round_B,round_C,round_D,round_E,round_F,round_G,round_H,fund_info_missing


status
operating    41829
Unknown       6170
acquired      3692
closed        2603
Name: count, dtype: int64

In [50]:
label='status'
time_limit=3600

In [51]:
predictor = TabularPredictor(label=label, problem_type='multiclass',eval_metric='f1_macro',).fit(
    train_data=df_final,
    time_limit=time_limit,
    verbosity=3
)

No path specified. Models will be saved in: "AutogluonModels/ag-20250429_090600"
Verbosity: 3 (Detailed Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.9.21
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 24.4.0: Fri Apr 11 18:34:14 PDT 2025; root:xnu-11417.101.15~117/RELEASE_ARM64_T8122
CPU Count:          8
GPU Count:          0
Memory Avail:       1.82 GB / 8.00 GB (22.7%)
Disk Space Avail:   13.91 GB / 228.27 GB (6.1%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'      

KeyboardInterrupt: 

In [52]:
predictor.leaderboard(df_final, silent=True)

AttributeError: module 'autogluon.tabular.predictor' has no attribute 'leaderboard'

In [25]:
predictor.evaluate(df_final, auxiliary_metrics=['accuracy'])

Loading: g:\PythonProjects\Pytorch_learning\Pytorch_learning\VC\AutogluonModels\ag-20250429_071012\models\LightGBM\model.pkl
Loading: g:\PythonProjects\Pytorch_learning\Pytorch_learning\VC\AutogluonModels\ag-20250429_071012\models\NeuralNetTorch\model.pkl
Loading: g:\PythonProjects\Pytorch_learning\Pytorch_learning\VC\AutogluonModels\ag-20250429_071012\models\WeightedEnsemble_L2\model.pkl


{'f1_macro': 0.764495599831698,
 'accuracy': 0.9094375069068406,
 'balanced_accuracy': 0.7238629106057288,
 'mcc': 0.7531324962917333}

In [ ]:
feature_data=predictor.feature_importance(df_final)

AttributeError: module 'autogluon.tabular.predictor' has no attribute 'feature_importance'